# Domain Scrape

In [124]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json
from pyspark.sql import SparkSession
import os

In [125]:
headers = {"User-Agent":
           "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

# home url of domian.com australia
home_url = "https://www.domain.com.au"

In [126]:
#configuration
postcode_range = [line.rstrip() for line in open('postcode.txt')]

#postcode of properties
postcode_list = [i for i in range(int(postcode_range[0]),int(postcode_range[1]))]

In [127]:
# return unique extracted link for given postcode
def link_by_postcode(postcode):
    
    # list to store all the urls of properties
    list_of_links = []
    
    # store unique link
    abc_links = [] 
    
    # number of pages of search result are 50
    page_numbers = list(range(50))[1:50]
    
    # for loop for all 50 search(melbourne region) pages
    for page in page_numbers:

        # extracting html document of search page
        url = home_url + f"/rent/melbourne-vic-{postcode}/?excludedeposittaken=1&page={page}"

        # parsing html document to 'lxml' format
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
        
        #break if current page not exist or no links found
        if ((bs_object is None) | (bs_object.find("ul", {"data-testid": "results"}) is None)):
            break
        
        #for each page, finding all the links available in 'ul' tag whos 'data-testid' is 'results'
        all_links = bs_object.find(
            "ul", {"data-testid": "results"}).findAll("a", href=re.compile("https://www.domain.com.au/*"))

        # inner loop to find links inside each property page because few properties are project so they have more properties inside their project page
        for link1 in all_links:
            # checking if it is a project and then performing similar above
            if 'project' in link1.attrs['href']:
                inner1_url = link1.attrs['href']
                inner1_bsobj = BeautifulSoup(requests.get(inner1_url, headers=headers).text, "html.parser")
                for link2 in inner1_bsobj.find("div", {"name": "listing-details__other-listings"}).findAll("a", href=re.compile("https://www.domain.com.au/*")):
                    if 'href' in link2.attrs:
                        list_of_links.append(link2.attrs['href'])
            else:
                list_of_links.append(link1.attrs['href'])
   
    # remove deplicated links
    for i in list_of_links: 
        if i not in abc_links: 
            abc_links.append(i) 
    return abc_links


In [128]:
def links_for_postcodes(postcode_list):
    all_links = []
    for postcode in postcode_list:
        current_postcode_links = link_by_postcode(postcode)
        all_links.extend(current_postcode_links)
    return list(set(all_links))
        

In [129]:
all_links = links_for_postcodes(postcode_list)

In [130]:
# defining required regural expression for data extraction     
pattern = re.compile(r'>(.+)<!.*>(.+?)</span>.*')
pattern1 = re.compile(r'>(.+)<.')
pattern2 = re.compile(r'destination=(.+)" rel=.')
#school
pattern_school = re.compile(r'>(.+)<')
pattern_distance = re.compile(r'">(.+)<!')
#street information
pattern_address1 = re.compile(r'>(.+)<!.*>')
pattern_address2 = re.compile(r'>(.+)<.')

In [131]:
basic_feature_list = []


# loop to iterate through each url
for link in all_links:
    
    # opening urls
    bs_object = BeautifulSoup(requests.get(link, headers=headers).text, "html.parser")
    
    # extracting address/name of property
    property_name = bs_object.find("h1", {"class": "css-164r41r"})
    
    # extracting baths, rooms, parking etc
    all_basic_features = bs_object.find("div", {"class": "css-ghc6s4"}).findAll("span", {"data-testid": "property-features-text-container"})
    
    # extracting property price
    property_price = bs_object.find("div", {"data-testid": "listing-details__summary-title"})
    
    # extracting latitudes and longitudes
    lat_long = bs_object.find("a", {"target": "_blank", 'rel': "noopener noreferer"})
    
    #extract schoolname, school distance
    # school= bs_object.find("div", {"data-testid": "listing-details__school-catchment"}).findAll("h5",{"class": "css-5w5cop"})
    school_data = json.loads(bs_object.find("script", {"id": "__NEXT_DATA__"}).text)
    school = [s['name'] for s in school_data['props']['pageProps']['componentProps']['schoolCatchment']['schools']]
    school_distance= [s['distance'] for s in school_data['props']['pageProps']['componentProps']['schoolCatchment']['schools']]
  
    # address info
    address1 = bs_object.find("span", {"data-testid": "address-line1"})
    address2 = bs_object.find("span", {"data-testid": "address-line2"})
    
    #recent rented
    recent_rented_number = bs_object.find("div", {"class": "css-12uo2x5"})
    
    
    #neighbourhood family vs single ratio
    neighb_family = bs_object.find("span", {"class": "css-1ruej2h"})
    neighb_single = bs_object.find("span", {"class": "css-1ymya4c"})
    
    
    #street owner vs renter
    street_renter = bs_object.find("span", {"class": "css-1ejmyle"})
    street_owner = bs_object.find("span", {"class": "css-1jhlvvv"})

    
    #long term resident percentage
    long_term_resi= bs_object.find("div", {"class": "css-ibsnk8"})
    
    
    # dictionary to store temporary data
    basic_feature_dict = {}
    
    # few properties does not contain all the 4 features such as rooms, baths, parkings, area. So need to check
    # how many features they contain
    if len(all_basic_features) == 4:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[2]))[0][1]] = pattern.findall(str(all_basic_features[2]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[3]))[0][1]] = pattern.findall(str(all_basic_features[3]))[0][0]
        
    elif len(all_basic_features) == 3:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[2]))[0][1]] = pattern.findall(str(all_basic_features[2]))[0][0]
        
    elif len(all_basic_features) == 2:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]
        basic_feature_dict[pattern.findall(str(all_basic_features[1]))[0][1]] = pattern.findall(str(all_basic_features[1]))[0][0]
        
    elif len(all_basic_features) == 1:
        basic_feature_dict[pattern.findall(str(all_basic_features[0]))[0][1]] = pattern.findall(str(all_basic_features[0]))[0][0]

        
        
    # putting 'none' if school is missing    
    if school is None:
        basic_feature_dict['school'] = None
        
    else:
        basic_feature_dict['school'] = school
    
    # putting 'none' if school distance is missing   
    if school_distance is None:
        basic_feature_dict['school_distance'] = None
        
    else:
        basic_feature_dict['school_distance'] = school_distance
     
    
    
    # putting 'none' if address(street) distance is missing   
    if address1 is None:
        basic_feature_dict['address1'] = None
        
    else:
        basic_feature_dict['address1'] = pattern_address1.findall(str(address1))
        
    # putting 'none' if address2 is missing   
    if address2 is None:
        basic_feature_dict['address2'] = None
        
    else:
        basic_feature_dict['address2'] = sum([pattern_address2.findall(str(i)) for i in address2],[])
    
    # putting 'none' if recent_rented number is missing   
    if recent_rented_number is None:
        basic_feature_dict['recent_rented_number'] = None
        
    else:
        basic_feature_dict['recent_rented_number'] = pattern1.findall(str(recent_rented_number))[0]
        
    
    
    # putting 'none' if longterm resident number is missing   
    if long_term_resi is None:
        basic_feature_dict['long_term_resi_percentage'] = None
        
    else:
        basic_feature_dict['long_term_resi_percentage']  = float(str(long_term_resi.text[:-1]))/100

    
    
    # putting 'none' if neighbourhood family/single number is missing   
    if (neighb_family is None) |(neighb_single is None) :
        basic_feature_dict['neighb_family_single_ratio'] = None
        
    else:
        #to avoid zero division
        add_on = 0.000001
        basic_feature_dict['neighb_family_single_ratio'] = float(str(neighb_family.text[:-1]))/(float(str(neighb_single.text[:-1]))+add_on)
        

        
    
    # putting 'none' if street owener/renter number is missing   
    if (street_owner is None) | (street_renter is None):
        basic_feature_dict['street_renter_owner_ratio'] = None
             
    else:
        #to avoid zero division
        add_on = 0.000001
        basic_feature_dict['street_renter_owner_ratio'] = float(str(street_renter.text[:-1]))/(float(str(street_owner.text[:-1]))+add_on)


        
    # putting 'none' if price is missing    
    if property_price is None:
        basic_feature_dict['price'] = None
        
    else:
        basic_feature_dict['price'] = pattern1.findall(str(property_price))[0]
        
    # putting 'none' if property name/address is missing       
    if property_name is None:
        basic_feature_dict['name'] = None
        
    else:
        basic_feature_dict['name'] = pattern1.findall(str(property_name))[0]
        
    # putting 'none' if latitude and logitude are missing        
    if lat_long is None:
        basic_feature_dict['lat'] = None
        basic_feature_dict['long'] = None
        
    else:
        basic_feature_dict['lat'] = pattern2.findall(str(lat_long))[0].split(',')[0]
        basic_feature_dict['long'] = pattern2.findall(str(lat_long))[0].split(',')[1]
        
    # appending all the data into a list
    basic_feature_list.append(basic_feature_dict)

In [132]:
# creating empty lists
beds_list = []
baths_list = []
parking_list = []
area_list = []
name_list = []
lat_list = []
long_list = []
price_list = []
school_list = []
school_distance_list =[]
address1_list = []
address2_list = []
recent_rented_number_list = []

street_renter_owner_ratio_list = []
neighb_family_single_ratio_list = []
long_term_resi_percentage_list =[]



# interating through list created above with data
for row in basic_feature_list:
    
    # checking if the row cointains 'Beds', 'Bed' or nothing
    if 'Beds' in row:
        beds_list.append(row['Beds'])
    elif 'bed' in row:
        beds_list.append(row['Bed'])
    else:
        beds_list.append(None)
        
    # checking if the row cointains 'Baths', 'Bath' or nothing    
    if 'Baths' in row:
        baths_list.append(row['Baths'])
    elif 'Bath ' in row:
        baths_list.append(row['Bath'])
    else:
        baths_list.append(None)
        
    # checking if the row cointains 'Parking', '-' or nothing     
    if 'Parking' in row and row['Parking'] != '−':
        parking_list.append(row['Parking'])
    else:
        parking_list.append(None)
        
    # checking if the row cointains ' ', or nothing. Because empty space (i.e. ' ') reprsents area  
    if ' ' in row:
        area_list.append(row[' '])
    else:
        area_list.append(None)
    # checking if the row cointains 'name' that is address of property         
    if 'name' in row:
        name_list.append(row['name'])
    else:
        name_list.append(None)
    
    # checking if the row cointains 'price'         
    if 'price' in row:
        price_list.append(row['price'])
    else:
        price_list.append(None)        
    
    # checking if the row cointains 'lat' that is lattitude of property         
    if 'lat' in row:
        lat_list.append(row['lat'])
    else:
        lat_list.append(None)  
        
    # checking if the row cointains 'long' that is lattitude of property             
    if 'long' in row:
        long_list.append(row['long'])
    else:
        long_list.append(None)
        
    # checking if the row cointains 'school' that is lattitude of property             
    if 'school' in row:
        school_list.append(row['school'])
    else:
        school_list.append(None)
      
    # checking if the row cointains 'school_distance'            
    if 'school_distance' in row:
        school_distance_list.append(row['school_distance'])
    else:
        school_distance_list.append(None)
        
    # checking if the row cointains 'address1'
    if 'address1' in row:
        address1_list.append(row['address1'])
    else:
        address1_list.append(None)
        
    # checking if the row cointains 'address2'      
    if 'address2' in row:
        address2_list.append(row['address2'])
    else:
        address2_list.append(None)
        
    # checking if the row cointains 'recent_rented_number'            
    if 'recent_rented_number' in row:
        recent_rented_number_list.append(row['recent_rented_number'])
    else:
        recent_rented_number_list.append(None)
    
        
    # checking if the row cointains 'long_term_resi_percentage'            
    if 'long_term_resi_percentage' in row:
        long_term_resi_percentage_list.append(row['long_term_resi_percentage'])
    else:
        long_term_resi_percentage_list.append(None)
        
    # checking if the row cointains 'street_renter_owner_ratio'            
    if 'street_renter_owner_ratio' in row:
        street_renter_owner_ratio_list.append(row['street_renter_owner_ratio'])
    else:
        street_renter_owner_ratio_list.append(None)
        
    # checking if the row cointains 'neighb_family_single_ratio'            
    if 'neighb_family_single_ratio' in row:
        neighb_family_single_ratio_list.append(row['neighb_family_single_ratio'])
    else:
        neighb_family_single_ratio.append(None)



        
        
        
        


In [133]:
#create dataframe 
house_dict = {}
house_dict['Beds'] = beds_list
house_dict['Baths'] = baths_list
house_dict['Parking'] = parking_list
house_dict['Area'] = area_list
house_dict['Address'] = name_list
house_dict['Latitude'] = lat_list
house_dict['Longitude'] = long_list
house_dict['Rent'] = price_list

house_dict['School'] = school_list
house_dict['School_distance'] = school_distance_list

house_dict['Location_1'] = address1_list
house_dict['Location_2'] = address2_list
house_dict['recent_rented_number'] = recent_rented_number_list



house_dict['street_renter_owner_ratio'] = street_renter_owner_ratio_list 
house_dict['neighb_family_single_ratio'] = neighb_family_single_ratio_list
house_dict['long_term_resi_percentage'] = long_term_resi_percentage_list


house_df = pd.DataFrame(house_dict)

In [134]:
def remove_chars(s):
    return re.sub('[^0-9.]+', '', s) 

def remove_squaremeter(s):
    if s != None:
        return remove_chars(s)
    else:
        return None

In [135]:
#find pattern for week/annual rent
year_pattern = r"p.a.|annually|annual"
week_pattern = r"pw|week"

regex_week_pattern = re.compile(week_pattern)
regex_year_pattern = re.compile(year_pattern)

#flag type of rent with 1(weekly), 0(annul), None(unknown)
def type_flag(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return 1
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return 0
    # not speficied
    else:
        return None

#convert rent to weekly rent
#if not sure rent type, it will be considered as weekly rent
def convert_rent(s):
    # week rent
    if (len(regex_week_pattern.findall(s.lower())) != 0 )& (len(regex_year_pattern.findall(s.lower())) == 0) :
        return remove_chars(s)
    # year rent
    elif (len(regex_week_pattern.findall(s.lower())) == 0) & (len(regex_year_pattern.findall(s.lower())) != 0) :
        return float(remove_chars(s))//12
    # not speficied
    else:
        return remove_chars(s)


house_df['Converted_Rent'] = [convert_rent(i) for i in price_list]
house_df['Type'] = [type_flag(i) for i in price_list]

In [136]:
#extract post code
house_df['Postcode'] = house_df['Address'].str[-4:]

#clean area
house_df['Area'] = [remove_squaremeter(i) for i in area_list]

In [137]:
house_df

,Beds,Baths,Parking,Area,Address,Latitude,Longitude,Rent,School,School_distance,Location_1,Location_2,recent_rented_number,street_renter_owner_ratio,neighb_family_single_ratio,long_term_resi_percentage,Converted_Rent,Type,Postcode
0,2,None,1,None,P08P/191 Powlett Street East Melbourne VIC 3002,-37.8100943,144.9860492,$625 per week,"[Collingwood College, Richmond West Primary Sc...","[880.9464005226209, 938.6647137000218, 1069.86...",None,None,None,NaN,0.818182,0.11,625,1.0,3002
1,None,None,None,None,806/238 Flinders street Melbourne VIC 3000,-37.8173927,144.9663866,$295 pw Furnished,"[Eltham College - Lonsdale Street Campus, Stot...","[737.9014649456448, 751.5165051029621, 866.835...",[238 Flinders St],"[Melbourne, VIC, 3000]",255,0.250000,0.204819,NaN,295,1.0,3000
2,None,None,None,None,6317/570 Lygon Street Carlton VIC 3053,-37.7931559,144.9694316,$300.00 per week,"[Carlton Primary School, Simonds Catholic Coll...","[339.18361293477284, 637.3712910204205, 878.73...",[570 Lygon St],"[Carlton, VIC, 3053]",669,0.176471,0.086957,0.05,300.00,1.0,3053
3,2,None,1,None,20/75 Drummond Street Carlton VIC 3053,-37.8047911,144.9673359,$650 per week,"[Carlton Gardens Primary School, Academy of Ma...","[359.8506074049445, 634.3270358541273, 750.340...",[75-91 Drummond St],"[Carlton, VIC, 3053]",54,3.000000,0.492537,0.26,650,1.0,3053
4,2,2,None,None,4916/228 La Trobe Street Melbourne VIC 3000,-37.8097514,144.9626141,$900/W Furnished,"[Eltham College - Lonsdale Street Campus, Stot...","[362.3625709073845, 729.545425869416, 787.3179...",[228 La Trobe St],"[Melbourne, VIC, 3000]",997,0.428571,0.176471,0.13,900,NaN,3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,None,None,None,None,4/29 Marine Parade St Kilda VIC 3182,-37.871783,144.9766561,$465 pw,"[Yesodei HaTorah College, St Kilda Primary Sch...","[622.3558352583495, 1035.4618345586548, 1255.9...",[29 Marine Pde],"[St Kilda, VIC, 3182]",10,0.428571,0.785714,0.34,465,1.0,3182
1426,None,None,None,None,1605B/639 Little Lonsdale Street Melbourne VIC...,-37.8141942,144.9521662,$580,"[Ozford College - Ozford College Campus, Ozfor...","[259.55083543938684, 260.5021570858975, 351.66...",None,None,None,NaN,0.250000,0.12,580,NaN,3000
1427,2,None,1,None,1008/77-89 A`Beckett Melbourne VIC 3000,-37.8090755,144.961809,$600 per week,"[Eltham College - Lonsdale Street Campus, Rive...","[404.9518921945805, 744.0790955751282, 761.667...",None,None,None,NaN,0.176471,0.13,600,1.0,3000
1428,2,2,1,None,2/1A Peel Street Windsor VIC 3181,-37.854585,144.9838292,$550 per week,"[Wesley College - St Kilda Road Junior Campus,...","[132.38805511626296, 655.7606252190784, 735.67...",[1a Peel St],"[Windsor, VIC, 3181]",49,1.000000,1.083333,0.52,550,1.0,3181


In [138]:
# save the property_df to parquet format
postcode_start = postcode_range[0]
postcode_end = postcode_range[1]


# check if it exists as it makedir will raise an error if it does exist
raw_output_dir = '../data/raw'

if not os.path.exists(raw_output_dir):
    os.makedirs(raw_output_dir)

In [139]:
# Create a spark session

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [140]:
filename = f'rental_info_{postcode_start}-{postcode_end}.parquet'
output_dir = f"{raw_output_dir}/{filename}"

In [141]:
house_df.to_parquet(output_dir)

In [23]:
pdf = spark.read.parquet('../data/raw/rental_info_3000-3100.parquet', headers=True)
pdf.limit(5)

Beds,Baths,Parking,Area,Address,Latitude,Longitude,Rent,School,School_distance,Location_1,Location_2,recent_rented_number,street_renter_owner_ratio,neighb_family_single_ratio,long_term_resi_percentage,Converted_Rent,Type,Postcode
null,null,1,null,105/10 Porter Str...,-37.8505434,144.9900254,$410 per week,[Prahran High Sch...,[164.341924489735...,[10 Porter St],"[Prahran, VIC, 3181]",60,0.4285714224489797,0.4925373060815328,0.16,410,1.0,3181
null,null,null,55,555 Flinders St M...,-37.820969,144.9564872,$430,[Hester Hornbrook...,[125.572861009537...,null,null,null,null,0.23456789833866792,0.05,430,null,3000
2,2,null,null,2402/155 Franklin...,-37.8088259,144.9595488,$750 **SPACIOUS A...,[Eltham College -...,[428.018069265046...,[155 Franklin St],"[Melbourne, VIC, ...",727,0.33333332888888895,0.38888888348765444,null,750,null,3000
2,2,null,null,2211/33 Rose Lane...,-37.8150137,144.9538694,$560 per week,[Ozford College -...,[253.084193694171...,[33 Rose La],"[Melbourne, VIC, ...",1007,0.4285714224489797,0.38888888348765444,0.12,560,1.0,3000
3,2,1,null,1202/7 Riverside ...,-37.8213341,144.9646982,$735 per week,[Hester Hornbrook...,[688.368276954260...,[Riverside Qy],"[Southbank, VIC, ...",842,0.6666666555555558,0.9230769053254442,0.23,735,1.0,3006


In [24]:
pdf.printSchema()

root
 |-- Beds: string (nullable = true)
 |-- Baths: string (nullable = true)
 |-- Parking: string (nullable = true)
 |-- Area: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Rent: string (nullable = true)
 |-- School: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- School_distance: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- Location_1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Location_2: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- recent_rented_number: string (nullable = true)
 |-- street_renter_owner_ratio: double (nullable = true)
 |-- neighb_family_single_ratio: double (nullable = true)
 |-- long_term_resi_percentage: double (nullable = true)
 |-- Converted_Rent: string (nullable = true)
 |-- Type: double (nullable = true)
 |-- Postcode: string (nulla

In [31]:
pdf.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Beds                        912 non-null    object 
 1   Baths                       398 non-null    object 
 2   Parking                     764 non-null    object 
 3   Area                        21 non-null     object 
 4   Address                     1431 non-null   object 
 5   Latitude                    1431 non-null   object 
 6   Longitude                   1431 non-null   object 
 7   Rent                        1431 non-null   object 
 8   School                      1431 non-null   object 
 9   School_distance             1431 non-null   object 
 10  Location_1                  1179 non-null   object 
 11  Location_2                  1179 non-null   object 
 12  recent_rented_number        1179 non-null   object 
 13  street_renter_owner_ratio   1179 

In [ ]:
"""
1. drop area
2. drop instance with null bed room
3. parking imputation, impute 0
4. baths imputation, mean? (see distribution)
5. check rent distribution -> check Type with None -> 
6. rremove extreme/unrealistic rent value


Schools
1. rating?

Location(street)
1. school/hospital/xxx on the street?

Postcode
1. give rating for each postcode?




"""